In [1]:
import numpy as np
import math
import random
import ast
import time
import random
import statistics as st

In [2]:
# Code taken from M. Freytsis & S. Schacht to generate Monte Carlo points
import numpy as np

class DKpp:
    ''' An implementation of the D -> K pi+ pi- decay amplitude,
        following code by S. Schacht.'''

    def __init__(self):
        ''' Load all of the numerical constants and preprocess them for quick access during evalution,'''
        # Some mass and width constants, using PDG 2016 values (in GeV)
        self.m = {'D0'  : 1.86483,
                  'K0'  : 0.497611,
                  'pic' : 0.13957018,
                  'pi0' : 0.1349766,
                  'eta' : 0.547862,
                  'etap': 0.95778}
        self.gam = {}
        self.L = {}
        self.Q = {}

        # pi-pi resonances
        self.m['rho770'],   self.gam['rho770'],   self.L['rho770']   = 0.77526, 0.1478,  1
        self.m['omega782'], self.gam['omega782'], self.L['omega782'] = 0.78265, 0.00849, 1
        self.m['f21270'],   self.gam['f21270'],   self.L['f21270']   = 1.2755,  0.1867,  2
        self.m['rho1450'],  self.gam['rho1450'],  self.L['rho1450']  = 1.465,   0.4,     1 #NB: Mass was 1000x too large in S. Schacht code

        # K-pi resonances
        self.m['Ks892'],    self.gam['Ks892'],    self.L['Ks892']    = 0.8937,  0.0472,  1 # Floating fit to data, not PDG value
        self.m['K2s1430'],  self.gam['K2s1430'],  self.L['K2s1430']  = 1.4256,  0.0985,  2
        self.m['Ks1680'],   self.gam['Ks1680'],   self.L['Ks1680']   = 1.717,   0.322,   1
        self.m['Ks1410'],   self.gam['Ks1410'],   self.L['Ks1410']   = 1.414,   0.232,   1 #NB: Mass was 1000x too large in S. Schacht code

        # Needed for seperate L=0 K-pi analysis
        self.m['K0s1430'],  self.gam['K0s1430']                      = 1.441,   0.193

    def M(self):
        ''' Return mass of Dalitz parent (for PS sampling).'''
        return self.m['D0']

    def m1(self):
        ''' Return mass of m1 (for PS sampling).'''
        return self.m['K0']

    def m2(self):
        ''' Return mass of m2 (for PS sampling).'''
        return self.m['pic']

    def m3(self):
        ''' Return mass of m3 (for PS sampling).'''
        return self.m['pic']

    def _pq(self, m12, m1, m2):
        ''' Helper function to compute daughter particle momentum is resonance rest frame.
            The bachelor momentum function is actually exactly the same, except m_1,2 = M, m_3
            Written in slightly different form than text above for efficent evaluation.'''
        facpp = m12 + m1 + m2
        facmm = m12 - m1 - m2
        facpm = m12 + m1 - m2
        facmp = m12 - m1 + m2

        return 0.5/m12 * np.sqrt(facpp * facmm * facpm * facmp, dtype='complex')

    def _rho(self, s, m1, m2):
        ''' Helper function for computing 2-particle phase-space factors for K-matrix formalism.'''
        return np.sqrt(1 + 0j - (m1+m2)**2/s)

    def _rhoapp(self, s):
        ''' Helper function for computing approximate phase-space matrix for 4 pi state in K-matrix formalism.'''
        return 1.0789*s**6 + 0.1366*s**5 - 0.2974*s**4 - 0.2084*s**3 + 0.1385*s**2 - 0.0193*s + 0.0005

    def sumampresonant(self, sKpm, sKpp, spp, components = False):
        '''The isobar ansatz for non-S-wave resonances.
           Fraction not used in calculation, but comes after-the-fact fit.'''
        # Resonance model specification, taken from [arXiv:1804.06153], Tables III
        resList = [{'id' : 'rho770',   'Q' : 0,
                    'coeff' : 1,                                       'frac'  : 0.204},  # coefficient fixed 
                   {'id' : 'omega782', 'Q' : 0,
                    'coeff' : 0.0388 * np.exp(1j * np.radians(120.7)), 'frac'  : 0.005},
                   {'id' : 'f21270',   'Q' : 0,
                    'coeff' : 1.43 * np.exp(1j * np.radians(-36.3)),   'frac'  : 0.008},
                   {'id' : 'rho1450',  'Q' : 0,
                    'coeff' : 2.85 * np.exp(1j * np.radians(102.1)),   'frac'  : 0.006},
                   {'id' : 'Ks892',    'Q' : -1,
                    'coeff' : 1.720 * np.exp(1j * np.radians(136.8)),  'frac'  : 0.599},
                   {'id' : 'K2s1430',  'Q' : -1,
                    'coeff' : 1.27 * np.exp(1j * np.radians(-44.1)),   'frac'  : 0.013},
                   {'id' : 'Ks1680',   'Q' : -1,
                    'coeff' : 3.31 * np.exp(1j * np.radians(-118.2)),  'frac'  : 0.005},
                   {'id' : 'Ks1410',   'Q' : -1,
                    'coeff' : 0.29 * np.exp(1j * np.radians(99.14)),   'frac'  : 0.001},
                   {'id' : 'Ks892',    'Q' : 1,
                    'coeff' : 0.164 * np.exp(1j * np.radians(-42.2)),  'frac'  : 0.006},
                   {'id' : 'K2s1430',  'Q' : 1,
                    'coeff' : 0.10 * np.exp(1j * np.radians(-89.6)),   'frac'  : 0.00},
                   {'id' : 'Ks1410',   'Q' : 1,
                    'coeff' : 0.21 * np.exp(1j * np.radians(150.2)),   'frac'  : 0.00}]

        # meson radius parameters (in 1/GeV)
        dD, dr = 5.0, 1.5

        if components:
            amp = []
        else:
            amp = np.zeros(len(sKpm), dtype='complex128')

        for res in resList:
            if res['Q'] == 0: #pi-pi resonances
                s = spp.astype(complex)
                p = self._pq(np.sqrt(s), self.m['pic'], self.m['pic'])
                p0 = self._pq(self.m[res['id']], self.m['pic'], self.m['pic'])
                q = self._pq(np.sqrt(s), self.m['D0'], self.m['K0'])
                pq = -0.5*(sKpp - self.m['pic']**2 - self.m['K0']**2 - 2*np.sqrt((self.m['pic']**2 + p**2)*(self.m['K0']**2 + q**2)))
                mD2, mC2 = self.m['D0']**2, self.m['K0']**2
                mB2, mA2 = self.m['pic']**2, self.m['pic']**2
                sAC, sBC = sKpp, sKpm
            elif res['Q'] == 1: #K-pi+ resonances
                s = sKpp.astype(complex)
                p = self._pq(np.sqrt(s), self.m['pic'], self.m['K0']) 
                p0 = self._pq(self.m[res['id']], self.m['pic'], self.m['K0'])
                q = self._pq(np.sqrt(s), self.m['D0'], self.m['pic'])
                pq = -0.5*(sKpm - self.m['pic']**2 - self.m['K0']**2 - 2*np.sqrt((self.m['K0']**2 + p**2)*(self.m['pic']**2 + q**2)))
                mD2, mC2 = self.m['D0']**2, self.m['pic']**2
                mB2, mA2 = self.m['K0']**2, self.m['pic']**2
                sAC, sBC = spp, sKpm
            elif res['Q'] == -1: #K-pi- responances
                s = sKpm.astype(complex)
                p = self._pq(np.sqrt(s), self.m['pic'], self.m['K0'])
                p0 = self._pq(self.m[res['id']], self.m['pic'], self.m['K0'])
                q = self._pq(np.sqrt(s), self.m['D0'], self.m['pic'])
                pq = -0.5*(sKpp - self.m['pic']**2 - self.m['K0']**2 - 2*np.sqrt((self.m['K0']**2 + p**2)*(self.m['pic']**2 + q**2)))
                mD2, mC2 = self.m['D0']**2, self.m['pic']**2
                mB2, mA2 = self.m['K0']**2, self.m['pic']**2
                sAC, sBC = spp, sKpp

            # Compute Blatt-Weisskopf barrier penetration factors and Zemach factors for the resonance
            z0, z = (p0*dr)**2, (p*dr)**2 #NB: factors were incorrectly p0*dr, p*dr in S. Schacht code
            if self.L[res['id']] == 1:
                Fr = np.sqrt( (1+z0)/(1+z) )
                Gamma = self.gam[res['id']] * (p/p0)**3 * self.m[res['id']]/np.sqrt(s) * Fr**2
                ZL = sAC - sBC + (mD2-mC2)*(mB2-mA2)/s #-2*pq
                #print('CLEO form: {}, Laura++ form: {}'.format(ZL, -4*pq))
            elif self.L[res['id']] == 2:
                Fr = np.sqrt( ((z0-3)**2+9*z0)/((z-3)**2+9*z) )
                Gamma = self.gam[res['id']] * (p/p0)**5 * self.m[res['id']]/np.sqrt(s) * Fr**2
                ZL = (sAC - sBC + (mD2-mC2)*(mB2-mA2)/s)**2 - 1/3 * (s - 2*(mD2+mC2) + (mD2-mC2)**2/s)*(s - 2*(mA2+mB2) + (mA2-mB2)**2/s) # 4/3 * (3*pq**2 - (p*q)**2)
                #print('CLEO form: {}, Laura++ form: {}'.format(ZL, 16/3 * (3*pq**2 - (p*q)**2)))

            # Compute propagator term for the resonance
            Tr = 1/(self.m[res['id']]**2 - s - 1j * self.m[res['id']] * Gamma)

            fullterm = res['coeff'] * Fr * ZL *  Tr
            if components:
                amp.append(fullterm)
            else:
                amp += fullterm

        return amp

    def F1(self, spp):
        '''The K-matrix model for pi-pi S-wave resonances.'''
        # Some parameterization constants, taken from [arXiv:1804.06153], Tables II & III
        # Represent couplings between pi-pi, K-K, 4pi, eta-eta, and eta-eta' channels
        mPoleKmat = np.array([0.65100, 1.20360, 1.55817, 1.21000, 1.82206])
        gKmat = np.array([[0.22889, -0.55377,  0.00000, -0.39899, -0.34639],
                          [0.94128,  0.55095,  0.00000,  0.39065,  0.31503],
                          [0.36856,  0.23888,  0.55639,  0.18340,  0.18681],
                          [0.33650,  0.40907,  0.85679,  0.19906, -0.00984],
                          [0.18171, -0.17558, -0.79658, -0.00355,  0.22358]])
        fscatt = np.array([0.23399, 0.15044, -0.20545, 0.32825, 0.35412])
        s0scatt, sA0, sA = -3.92637, -0.15, 1.
        
        # Only 4 entries because production of eta-eta' channel in D decays not kinematically possible (I think)
        beta = np.array([8.5 * np.exp(1j * np.radians(68.5)), 12.2 * np.exp(1j * np.radians(24.0)),
                         29.2 * np.exp(1j * np.radians(-0.1)), 10.8 * np.exp(1j * np.radians(-51.9))])
        fprod = np.array([8.0 * np.exp(1j * np.radians(-126.0)), 26.3 * np.exp(1j * np.radians(-152.3)),
                          33.0 * np.exp(1j * np.radians(-93.2)), 26.2 * np.exp(1j * np.radians(-121.4)),0])
        s0prod = -0.07 # fixed as a reference point

        fA0 = (1 - sA0)/(spp - sA0) * (spp - sA * 0.5 * self.m['pic']**2)

        # Create production vector P 
        P = fprod.reshape(1,-1) * ((1 - s0prod)/(spp - s0prod)).reshape(-1,1) # add dummy dimensions for broadcasting
        for i, bfac in enumerate(beta):
            P += bfac * gKmat[i].reshape(1,-1)/(mPoleKmat[i]**2 - spp.reshape(-1,1))

        # Create the K matrix
        fmat = np.zeros((5,5), dtype='complex128')
        fmat[0,:] = fscatt

        K = fmat[np.newaxis, :] * ((1 - s0scatt)/(spp - s0scatt) * fA0)[:, np.newaxis, np.newaxis]
        for i, m in enumerate(mPoleKmat):
            K += np.outer(gKmat[i],gKmat[i])[np.newaxis, :] * (fA0/(m**2 - spp))[:, np.newaxis, np.newaxis]
        
        # Create phase-space matrix, phase space functions taken from [arXiv:1711.09854]
        rho = np.zeros((len(spp),5,5), dtype='complex128')
        rho[:,0,0] = self._rho(spp, self.m['pic'], self.m['pic'])
        rho[:,1,1] = self._rho(spp, self.m['K0'], self.m['K0'])
        rho[:,2,2] = np.where(spp < 1, self._rhoapp(spp), self._rho(spp, 2*self.m['pic'], 2*self.m['pic']))
        rho[:,3,3] = self._rho(spp, self.m['eta'], self.m['eta'])
        rho[:,4,4] = self._rho(spp, self.m['eta'], self.m['etap'])
        
        # Put everything together

        result = np.squeeze(np.linalg.inv(np.eye(5)[np.newaxis, :] - 1j * K @ rho) @ P[np.newaxis, :, :, np.newaxis], axis=(0,3))

        return result[:,0] # Take the pi-pi component

    def KpiSwaveamp(self, sKp):
        '''The LASS parameterization for K-pi S-wave resonances,'''
        # Some parameterization constants, taken from [arXiv:1804.06153], Table III
        F, R = 0.96, 1                                   # R is fixed as reference point
        phiF, phiR = np.radians(0.1), np.radians(-109.7)
        a, r = 0.113, -33.8                              # scattering length/effective range

        # Spectator momentum in resonance rest frame
        q = self._pq(np.sqrt(sKp), self.m['pic'], self.m['K0'])       # FIXME: I suspect this is wrong and needs to use D & pi masses (p <> q issue)
        q0 = self._pq(self.m['K0s1430'], self.m['pic'], self.m['K0']) # FIXME: Ditto
        Gamma = self.gam['K0s1430'] * (q/q0) * self.m['K0s1430']/np.sqrt(sKp)

        deltaR = phiR + np.arctan(self.m['K0s1430']*Gamma/(self.m['K0s1430']**2 - sKp))
        #deltaF = phiF + np.arctan2(1, 1/(a*q) + (r*q)/2)              # Rewriting of arccot via 2-argument arctan
        deltaF = phiF + np.arctan(2*a*q/(2+a*q**2*r))              # Rewriting of arccot via arctan

        amp = R * np.sin(deltaR) * np.exp(1j*deltaR) * np.exp(2j*deltaF) + F * np.sin(deltaF) * np.exp(1j*deltaF)

        return amp


    def full(self,DalitzList):
        s12 = DalitzList[:, 0]
        s13 = DalitzList[:, 1]
        s23 = self.m['D0']**2 + self.m['K0']**2 + 2*self.m['pic']**2 - s12 - s13 # cf. [arXiv:1711.09854], eq. 1

        # The coefficients of the K-pi S-wave ampltitudes are missing from [arXiv:1804.06153] equations,
        # but present in the fit tables, so hopefully my guess at the form is correct.
        amp = self.sumampresonant(s13, s12, s23) + self.F1(s23) \
              + 2.36 * np.exp(1j * np.radians(99.4)) * self.KpiSwaveamp(s13) \
              + 0.11 * np.exp(1j * np.radians(162.3)) * self.KpiSwaveamp(s12)

        return amp

    def decomp(self, DalitzList):
        amp = self.full(DalitzList)

        mag2 = np.abs(amp)**2
        phase = np.angle(amp)

        return (mag2, phase)

class BKpp(DKpp):
    ''' An implementation of the B -> (K pi+ pi-) K decay amplitudes.'''

    def __init__(self, rB = 0.0994, deltaB = None, gamma = None, Bcharge=+1): # Central values taken from HFLAV 2022 world averages
        ''' Load all standard D amplitude funcitonality, along with B -> DK parameters.'''
        if deltaB == None:
            deltaB = np.radians(127.7)
        if gamma == None:
            gamma = np.radians(65.9)

        super().__init__()

        self.rB = rB
        self.deltaB = deltaB
        self.gamma = gamma
        self.Bcharge = Bcharge

    # This now matches the convention arXiv:2211.05133.
    def fullBp(self, DalitzList):
        ''' Amplitude for B+ decays.'''
        amp = super().full(DalitzList[:,::-1]) + self.rB * np.exp(1j*(self.deltaB + self.gamma)) * super().full(DalitzList)

        return amp

    def fullBm(self, DalitzList):
        ''' Amplitude for B- decays.'''
        amp = super().full(DalitzList) + self.rB * np.exp(1j*(self.deltaB - self.gamma)) * super().full(DalitzList[:,::-1])

        return amp

    def full(self, DalitzList):
        ''' Compatability with AmpSample.'''
        if self.Bcharge == +1:
            return self.fullBp(DalitzList)
        if self.Bcharge == -1:
            return self.fullBm(DalitzList)

class DalitzSample:
    ''' A class for efficient uniform sampling of the Dalitz plane. '''

    def __init__(self, amp):
        ''' Initialize all Dalitz plane information.
            NB: requires amplitude to be able to provide mother and daughter particle masses.'''
        self.M = amp.M()
        self.m1, self.m2, self.m3 = amp.m1(), amp.m2(), amp.m3()

    def generate(self, nsamples):
        ''' Generate kinematically allowed points uniformly in the s12-s13 plane.'''
        finalpoints = np.zeros((nsamples,2))
        
        samplesleft = nsamples

        # generate samples with rejection of unphysical points until full result is filled
        while samplesleft > 0:
            points = np.random.rand(samplesleft, 2)
            
            points[:,0] = ((self.M - self.m3)**2 - (self.m1 + self.m2)**2)*points[:,0] + (self.m1 + self.m2)**2 #s12
            points[:,1] = ((self.M - self.m2)**2 - (self.m1 + self.m3)**2)*points[:,1] + (self.m1 + self.m3)**2 #s13
            
            # Check if points are kinematically valid (from PDG kinematics review)
            E1star2 = (points[:,0] - self.m2**2 + self.m1**2)**2/(4*points[:,0])
            E3star2 = (self.M**2 - points[:,0] - self.m3**2)**2/(4*points[:,0])
            E1E3star = (points[:,0] - self.m2**2 + self.m1**2)*(self.M**2 - points[:,0] - self.m3**2)/(4*points[:,0])
            
            s13max = (E1star2 + 2*E1E3star + E3star2) - (np.sqrt(E1star2 - self.m1**2) - np.sqrt(E3star2 - self.m3**2))**2
            s13min = (E1star2 + 2*E1E3star + E3star2) - (np.sqrt(E1star2 - self.m1**2) + np.sqrt(E3star2 - self.m3**2))**2

            validpoints = points[(points[:,1] < s13max) & (points[:,1] > s13min)]

            first = nsamples-samplesleft
            finalpoints[first:first+len(validpoints),:] = validpoints
            samplesleft -= len(validpoints)

        return finalpoints

class AmpSample(DalitzSample):
    ''' A class to produce weighted and unweighted Dalitz plane events. '''

    def __init__(self, amp):
        self.amp = amp
        self.ps = DalitzSample(amp)

        # Mainly for plotting purposes
        self.M = amp.M()
        self.m1, self.m2, self.m3 = amp.m1(), amp.m2(), amp.m3()

    def generate_weighted(self, nsamples):
        ''' Generate weighted samples.
            This is just flat PS sampling with |amp|^2 weights appended.'''
        
        points = self.ps.generate(nsamples)

        amplitudes  = self.amp.full(points)
        weights = amplitudes.real**2 + amplitudes.imag**2 # About x10 faster than computing |amp|^2
        #phases = np.arctan( amplitudes.imag / amplitudes.real)
        #phases[phases < 0] += 2*np.pi

        #return points, weights, phases
        return points, weights

    def generate(self, nsamples, nbatch=5000):
        ''' Generate unweighted samples.
            This keeps generating test points in batches of nbatch until enough
            events have been produced. Not very efficient, but good enough for the job. '''

        # Initialize bookkeeping
        npass = 0
        maxweight = 0.
        pointList = []
        weightList = []
        acceptList = []

        while npass < nsamples:
            # Generate points & weights
            pointList.append(self.ps.generate(nbatch))
            amplitude = self.amp.full(pointList[-1])
            weightList.append(amplitude.real**2 + amplitude.imag**2)

            # Check if maximum weight needs to be updated
            if weightList[-1].max() > maxweight:
                maxweight = weightList[-1].max()
                for i, weights in enumerate(weightList[:-1]):
                    acceptList[i] = (np.random.rand(nbatch) < weights/maxweight)

            acceptList.append(np.random.rand(nbatch) < weightList[-1]/maxweight)

            npass = sum([accept.sum() for accept in acceptList])

        # Package final accepted points together
        finalpoints = np.concatenate([points[accept] for accept, points in zip(acceptList, pointList)])

        return finalpoints[:nsamples,:]

In [3]:
start = time.time()

# Insert scenario(s) here.

rBTrue = 0.9
deltaBDegTrue = 60
gammaDegTrue = 150

DSampler = AmpSample(DKpp())
BPlusSampler = AmpSample(BKpp(rB = rBTrue, deltaB = np.radians(deltaBDegTrue), gamma = np.radians(gammaDegTrue), Bcharge = +1))
BMinusSampler = AmpSample(BKpp(rB = rBTrue, deltaB = np.radians(deltaBDegTrue), gamma = np.radians(gammaDegTrue), Bcharge = -1))

spacing = 0.01
countTot = 1000
typ = 'none' # can be either 'none', 'tilde', or 'bar'

sList = []
s = 0.35
while s <= 3.0:
    sList.append([s])
    s = s + spacing
sLength = len(sList)
sList = np.array(sList)

In [4]:
# Computes N(s12, s13) or N(s13, s12) 
#(with a tilde too) for either B^+, B^-, or D.
# Possibilities for var: 'none', 'tilde', 'bar'.
def count_events(events, flip=False, var='none'):
    N = []
    eventsXLong = np.tile(events[:,0],(sLength,1,))
    eventsYLong = np.tile(events[:,1],(sLength,1,))
    
    if (var == 'none'):
        eventsX = eventsXLong < sList
        eventsY = eventsYLong < sList
        
        if not flip:
            for i in range(sLength):
                for j in range(sLength):
                    count = np.count_nonzero(np.logical_and(eventsX[i], eventsY[j]))
                    N.append(count)
        else:
            for i in range(sLength):
                for j in range(sLength):
                    count = np.count_nonzero(np.logical_and(eventsX[j], eventsY[i]))
                    N.append(count)
    
    elif (var == 'tilde'):
        eventsX = eventsXLong > sList
        eventsY = eventsYLong > sList   
        
        if not flip:
            for i in range(sLength):
                for j in range(sLength):
                    count = np.count_nonzero(np.logical_and(eventsX[i], eventsY[j]))
                    N.append(count)
        else:
            for i in range(sLength):
                for j in range(sLength):
                    count = np.count_nonzero(np.logical_and(eventsX[j], eventsY[i]))
                    N.append(count)
                    
    elif (var == 'bar'):
        eventsX = eventsXLong > sList
        eventsY = eventsYLong < sList   
        
        if not flip:
            for i in range(sLength):
                for j in range(sLength):
                    count = np.count_nonzero(np.logical_and(eventsX[i], eventsY[j]))
                    N.append(count)
        else:
            for i in range(sLength):
                for j in range(sLength):
                    count = np.count_nonzero(np.logical_and(eventsX[j], eventsY[i]))
                    N.append(count)
    
    return N


# Computes R_sigmaS, R_sigmaA, R_deltaS, R_deltaA for each [s12, s13]
# (tilde included also).
# Possibilities for var: 'none', 'tilde', 'bar'.
def compute_R(validPointsPlusInt, validPointsMinusInt, var='none'):
    
    NPlusNoFlip = np.array(count_events(validPointsPlusInt, var=var))
    NPlusFlip = np.array(count_events(validPointsPlusInt, flip=True, var=var))
    NMinusNoFlip = np.array(count_events(validPointsMinusInt, var=var))
    NMinusFlip = np.array(count_events(validPointsMinusInt, flip=True, var=var))
    
    sigmaPlus = 1/2 * (NPlusNoFlip + NPlusFlip)
    sigmaMinus  = 1/2 * (NMinusNoFlip + NMinusFlip)
    deltaPlus = 1/2 * (NPlusNoFlip - NPlusFlip)
    deltaMinus = 1/2 * (NMinusNoFlip - NMinusFlip)
    
    RSigmaS = 1/2 * (sigmaPlus + sigmaMinus)
    RSigmaA = 1/2 * (sigmaPlus - sigmaMinus)  
    RDeltaS = 1/2 * (deltaPlus + deltaMinus)
    RDeltaA = 1/2 * (deltaPlus - deltaMinus)
        
    return [RSigmaS, RSigmaA, RDeltaS, RDeltaA, NPlusNoFlip, NPlusFlip, NMinusNoFlip, NMinusFlip]

In [ ]:
minGammas = []
minDeltaBs = []
minDs = []
minRbs = []
count = 1

# Stefan computed these values.
rPlus = 0.5524859930147382
rMinus = 0.5524863826760069

while count <= countTot:
    
    validPointsMinus = np.array(BMinusSampler.generate(nsamples = 6436, nbatch = 50000))
    validPointsPlus = np.array(BPlusSampler.generate(nsamples = 6733, nbatch = 50000))
    validPointsD = np.array(DSampler.generate(5575, nbatch=50000))
    
    NPlusTot = len(validPointsPlus)
    NMinusTot = len(validPointsMinus)
    NDTot = len(validPointsD)

    Rs = compute_R(validPointsPlus, validPointsMinus, var=typ)

    NDNoFlip = np.array(count_events(validPointsD, var=typ))
    NDFlip = np.array(count_events(validPointsD, var=typ, flip=True))

    RDeltaS = Rs[2]
    RSigmaA = Rs[1]

    NPlusNoFlip = Rs[4]
    NPlusFlip = Rs[5]
    NMinusNoFlip = Rs[6]
    NMinusFlip = Rs[7]

    # Lattice spacings
    deltaBDegSpacing = 2
    gammaDegSpacing = 2
    rBSpacing = 0.01
    
    minGammasTemp = []
    minDeltaBsTemp = []
    minDsTemp = []

    rB = 0.80
    while rB <= 1.00:
        minGammasTempTemp = []
        minDsTempTemp = []
        
        deltaBDeg = 1
        while deltaBDeg < 90:

            deltaBRad = math.radians(deltaBDeg)

            RDeltaSWGamma = []
            RSigmaAWGamma = []

            gammaDeg = 91
            while gammaDeg < 180:

                gammaRad = math.radians(gammaDeg)
                RDeltaSWGamma.append(RDeltaS)
                a = -(math.cos(deltaBRad) / math.sin(deltaBRad)) * (math.cos(gammaRad) / math.sin(gammaRad))
                RSigmaAWGamma.append(a * RSigmaA)

                gammaDeg = gammaDeg + gammaDegSpacing

            RDeltaSWGamma = np.array(RDeltaSWGamma)
            RSigmaAWGamma = np.array(RSigmaAWGamma)

            RSigmaSSubWGamma = []
            RDeltaASubWGamma = []
            
            i = 0
            while i < 45:

                NPlusSubNoFlip = NPlusNoFlip - rPlus*(NPlusTot / NDTot)*(NDFlip + (rB**2)*NDNoFlip)
                NMinusSubNoFlip = NMinusNoFlip - rMinus*(NMinusTot / NDTot)*(NDNoFlip + (rB**2)*NDFlip)
                NPlusSubFlip = NPlusFlip - rPlus*(NPlusTot / NDTot)*(NDNoFlip + (rB**2)*NDFlip)
                NMinusSubFlip = NMinusFlip - rMinus*(NMinusTot / NDTot)*(NDFlip + (rB**2)*NDNoFlip)

                SigmaPlusSub = 1/2 * (NPlusSubNoFlip + NPlusSubFlip)
                SigmaMinusSub = 1/2 * (NMinusSubNoFlip + NMinusSubFlip)
                DeltaPlusSub = 1/2 * (NPlusSubNoFlip - NPlusSubFlip)
                DeltaMinusSub = 1/2 * (NMinusSubNoFlip - NMinusSubFlip)

                RSigmaSSub = 1/2 * (SigmaPlusSub + SigmaMinusSub)
                RDeltaASub = 1/2 * (DeltaPlusSub - DeltaMinusSub)

                RSigmaSSubWGamma.append(RSigmaSSub)

                gammaRad = math.radians(int(gammaDegSpacing*i + 91))
                b = (math.cos(deltaBRad) / math.sin(deltaBRad)) * (math.sin(gammaRad) / math.cos(gammaRad))

                RDeltaASubWGamma.append(b * RDeltaASub)

                i = i + 1

            RSigmaSSubWGamma = np.array(RSigmaSSubWGamma)
            RDeltaASubWGamma = np.array(RDeltaASubWGamma)

            DSigma = np.absolute(RSigmaSSubWGamma - RSigmaAWGamma)
            DDelta = np.absolute(RDeltaSWGamma - RDeltaASubWGamma)

            DSigmaMax = []
            DDeltaMax = []

            i = 0
            while i < 45:
                DSigmaMax.append(max(DSigma[i]))
                DDeltaMax.append(max(DDelta[i]))

                i = i + 1

            minSigmaPred = int(gammaDegSpacing*DSigmaMax.index(min(DSigmaMax)) + 91)
            minDeltaPred = int(gammaDegSpacing*DDeltaMax.index(min(DDeltaMax)) + 91)

            DSigmaMax = np.array(DSigmaMax)
            DDeltaMax = np.array(DDeltaMax)

            D = DSigmaMax + DDeltaMax
            D = D.tolist()

            DSigmaMax = DSigmaMax.tolist()
            DDeltaMax = DDeltaMax.tolist()

            minDPred = int(gammaDegSpacing*D.index(min(D)) + 91)
            minimumD = min(D)
            
            minGammasTempTemp.append(minDPred)
            minDsTempTemp.append(minimumD)
            
            deltaBDeg = deltaBDeg + deltaBDegSpacing
            
        minimumDFixedRB = min(minDsTempTemp)                
        indexOfMinDFixedRB = minDsTempTemp.index(min(minDsTempTemp))
        
        minDsTemp.append(minimumDFixedRB)
        minGammasTemp.append(minGammasTempTemp[indexOfMinDFixedRB])
        minDeltaBsTemp.append(int(deltaBDegSpacing*indexOfMinDFixedRB + 1))
        
        rB = round(rB + rBSpacing, 2)
    
    minimumDEverythingVaried = min(minDsTemp)
    indexOfMinDEverythingVaried = minDsTemp.index(min(minDsTemp))
    
    minDs.append(minimumDEverythingVaried)
    minGammas.append(minGammasTemp[indexOfMinDEverythingVaried])
    minDeltaBs.append(minDeltaBsTemp[indexOfMinDEverythingVaried])
    minRbs.append(round(indexOfMinDEverythingVaried*rBSpacing + 0.80, 2))
        
    print("D Minimum Value: " + str(minDs[-1]))
    print("Gamma: " + str(minGammas[-1]))
    print("DeltaB: " + str(minDeltaBs[-1]))
    print("rB: " + str(minRbs[-1]))
    
    print('Sample number ' + str(count))
    print('List of minimum test statistics thus far: ' + str(minDs))
    print('List of gamma predictions thus far: ' + str(minGammas))
    print('List of deltaB predictions thus far: ' + str(minDeltaBs))
    print('List of rB predictions thus far: ' + str(minRbs))
            
    end1 = time.time()
    timeElapsedMins1 = (end1 - start) / 60
    print(timeElapsedMins1)
    
    count = count + 1
        
    print('-------------------------------------------------')    

print('-------------------------------------------------')
end = time.time()
timeElapsedMins = (end - start) / 60
print('Time elapsed: ' + str(timeElapsedMins) + ' mins')

D Minimum Value: 171.09320542722753
Gamma: 153
DeltaB: 59
rB: 0.86
Sample number 1
List of minimum test statistics thus far: [171.09320542722753]
List of gamma predictions thus far: [153]
List of deltaB predictions thus far: [59]
List of rB predictions thus far: [0.86]
11.45551648537318
-------------------------------------------------
D Minimum Value: 135.46238855025035
Gamma: 155
DeltaB: 63
rB: 0.87
Sample number 2
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035]
List of gamma predictions thus far: [153, 155]
List of deltaB predictions thus far: [59, 63]
List of rB predictions thus far: [0.86, 0.87]
22.656309469540915
-------------------------------------------------
D Minimum Value: 130.27266677748054
Gamma: 151
DeltaB: 57
rB: 0.87
Sample number 3
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054]
List of gamma predictions thus far: [153, 155, 151]
List of deltaB predictions thus far: [59, 63, 57]


D Minimum Value: 116.08814098249705
Gamma: 145
DeltaB: 63
rB: 0.89
Sample number 16
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705]
List of gamma predictions thus far: [153, 155, 151, 143, 145, 157, 151, 155, 149, 147, 145, 149, 147, 147, 155, 145]
List of deltaB predictions thus far: [59, 63, 57, 61, 61, 57, 61, 63, 61, 63, 57, 61, 55, 65, 65, 63]
List of rB predictions thus far: [0.86, 0.87, 0.87, 0.88, 0.88, 0.86, 0.88, 0.87, 0.88, 0.88, 0.87, 0.87, 0.88, 0.88, 0.87, 0.89]
179.09173626899718
-------------------------------------------------
D Minimum Value: 125.48084449749562
Gamma: 157
DeltaB: 63
rB: 0.87
Sample number 17
List of minimum test statistics thus far: [171.09320542

D Minimum Value: 118.14907769374825
Gamma: 137
DeltaB: 61
rB: 0.88
Sample number 25
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825]
List of gamma predictions thus far: [153, 155, 151, 143, 145, 157, 151, 155, 149, 147, 145, 149, 147, 147, 155, 145, 157, 161, 155, 161, 155, 165, 143, 145, 137]
List of deltaB predictions thus far: [59, 63, 57, 61, 61, 57, 61, 63, 61, 63, 57, 61, 55, 65, 65, 63, 63, 65, 65, 63, 59, 69, 65, 59, 61]
List of rB predictions thus far: [0.86, 0.87, 0.87, 0.88, 0.88, 0.86, 0.88, 0.87, 

D Minimum Value: 103.04502736898073
Gamma: 149
DeltaB: 59
rB: 0.87
Sample number 32
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825, 135.01803096354408, 132.4971870741859, 115.91528488544614, 123.17042680982283, 134.70644852263945, 130.40284816836055, 103.04502736898073]
List of gamma predictions thus far: [153, 155, 151, 143, 145, 157, 151, 155, 149, 147, 145, 149, 147, 147, 155, 145, 157, 161, 155, 161, 155, 165, 143, 145, 137, 145, 153, 147, 149, 151, 149, 149]
List of deltaB predictions thus far: [59, 63, 

D Minimum Value: 131.68568780708415
Gamma: 149
DeltaB: 57
rB: 0.88
Sample number 38
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825, 135.01803096354408, 132.4971870741859, 115.91528488544614, 123.17042680982283, 134.70644852263945, 130.40284816836055, 103.04502736898073, 109.67031910588982, 143.50337546899075, 116.99169283550651, 114.33024195924142, 116.30475733570255, 131.68568780708415]
List of gamma predictions thus far: [153, 155, 151, 143, 145, 157, 151, 155, 149, 147, 145, 149, 147, 147, 155, 145, 157, 1

...

D Minimum Value: 100.32093148485286
Gamma: 151
DeltaB: 61
rB: 0.87
Sample number 243
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825, 135.01803096354408, 132.4971870741859, 115.91528488544614, 123.17042680982283, 134.70644852263945, 130.40284816836055, 103.04502736898073, 109.67031910588982, 143.50337546899075, 116.99169283550651, 114.33024195924142, 116.30475733570255, 131.68568780708415, 112.92922445289491, 136.82608431301836, 140.26216672503415, 143.11363316085556, 102.00264120950705, 114.72914611406249, 17

D Minimum Value: 134.6848611324047
Gamma: 151
DeltaB: 69
rB: 0.89
Sample number 244
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825, 135.01803096354408, 132.4971870741859, 115.91528488544614, 123.17042680982283, 134.70644852263945, 130.40284816836055, 103.04502736898073, 109.67031910588982, 143.50337546899075, 116.99169283550651, 114.33024195924142, 116.30475733570255, 131.68568780708415, 112.92922445289491, 136.82608431301836, 140.26216672503415, 143.11363316085556, 102.00264120950705, 114.72914611406249, 175

D Minimum Value: 104.44193215274217
Gamma: 145
DeltaB: 65
rB: 0.88
Sample number 245
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825, 135.01803096354408, 132.4971870741859, 115.91528488544614, 123.17042680982283, 134.70644852263945, 130.40284816836055, 103.04502736898073, 109.67031910588982, 143.50337546899075, 116.99169283550651, 114.33024195924142, 116.30475733570255, 131.68568780708415, 112.92922445289491, 136.82608431301836, 140.26216672503415, 143.11363316085556, 102.00264120950705, 114.72914611406249, 17

D Minimum Value: 136.5727912554159
Gamma: 163
DeltaB: 67
rB: 0.87
Sample number 246
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825, 135.01803096354408, 132.4971870741859, 115.91528488544614, 123.17042680982283, 134.70644852263945, 130.40284816836055, 103.04502736898073, 109.67031910588982, 143.50337546899075, 116.99169283550651, 114.33024195924142, 116.30475733570255, 131.68568780708415, 112.92922445289491, 136.82608431301836, 140.26216672503415, 143.11363316085556, 102.00264120950705, 114.72914611406249, 175

D Minimum Value: 110.06410039661122
Gamma: 143
DeltaB: 61
rB: 0.89
Sample number 247
List of minimum test statistics thus far: [171.09320542722753, 135.46238855025035, 130.27266677748054, 93.79236829340076, 108.33198568246459, 143.26872508148676, 104.40036901803012, 97.19131767532004, 119.49393025860618, 94.86713622979038, 149.79572420270404, 117.1835233945274, 120.99633061934533, 119.83050287454645, 139.19756399253373, 116.08814098249705, 125.48084449749562, 147.6029934109527, 132.19427018843749, 129.014882624971, 147.05662487191378, 160.1574824108029, 97.25638257934591, 104.59457955407956, 118.14907769374825, 135.01803096354408, 132.4971870741859, 115.91528488544614, 123.17042680982283, 134.70644852263945, 130.40284816836055, 103.04502736898073, 109.67031910588982, 143.50337546899075, 116.99169283550651, 114.33024195924142, 116.30475733570255, 131.68568780708415, 112.92922445289491, 136.82608431301836, 140.26216672503415, 143.11363316085556, 102.00264120950705, 114.72914611406249, 17

In [ ]:
# Error computations for 1000 samples.

minGammas.sort()
minDeltaBs.sort()
minRbs.sort()

print(minGammas)

centerGamma = st.mean(minGammas)
leftErrorGamma = centerGamma - minGammas[160]
rightErrorGamma = minGammas[-161] - centerGamma

print(centerGamma)
print(leftErrorGamma)
print(rightErrorGamma)

print('-------------------------------------------------')

print(minDeltaBs)

centerDeltaB = st.mean(minDeltaBs)
leftErrorDeltaB = centerDeltaB - minDeltaBs[160]
rightErrorDeltaB = minDeltaBs[-161] - centerDeltaB

print(centerDeltaB)
print(leftErrorDeltaB)
print(rightErrorDeltaB)

print('-------------------------------------------------')

print(minRbs)

centerRb = st.mean(minRbs)
leftErrorRb = centerRb - minRbs[160]
rightErrorRb = minRbs[-161] - centerRb

print(centerRb)
print(leftErrorRb)
print(rightErrorRb)